# Image Classification using CNN

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# parameters
batch_size = 100
learning_rate = 1e-4
training_epochs = 2 #10

In [ ]:
# layer functions
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1],
                              padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                             strides=[1,2,2,1],
                             padding='SAME')

In [ ]:
# MNIST dataset
mnist = input_data.read_data_sets("./MNIST_DATA", one_hot=True)

In [ ]:
# input
X = tf.placeholder(tf.float32, shape=[None, 784])
Y_ = tf.placeholder(tf.float32, shape=[None, 10])
# reshape (N, Height, Width, Channels)
X_image = tf.reshape(X, [-1, 28, 28, 1])

# dropout
keep_prob = tf.placeholder(tf.float32)

In [ ]:
# ConvNet-1
# 28x28x1 -> 5x5x32 filter -> 2x2 max_pooling -> 14x14x32
# Weight [Height, Width, IN_Channels, OUT_Channels]
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(X_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [ ]:
# ConvNet-2
# 14x14x32 -> 5x5x64 filter -> 2x2 max_pooling -> 7x7x64
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [ ]:
# FC-1
# 7x7x64 -> Flattening -> 1x1x1024
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [ ]:
# Output
# 1024 -> 10
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
Y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
# Loss (Cross Entropy) 
cross_entropy = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(
                        logits=Y, labels=Y_))

# Optimize (Adam)
train_step = tf.train.AdamOptimizer(
                    learning_rate).minimize(cross_entropy)

In [ ]:
# accuracy
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
%%time
# train
for epoch in range(training_epochs):
    batch_count = int(mnist.train.num_examples/batch_size)
    for i in range(batch_count):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _, acc = sess.run([train_step, accuracy], 
                             feed_dict={X: batch_x,
                                        Y_: batch_y,
                                        keep_prob: 0.5})
    print("Epoch: ", epoch, "Train Accuracy: ", acc)

In [ ]:
%%time
# Test
print("Test Accuracy: ", sess.run(accuracy,
                                 feed_dict={X: mnist.test.images,
                                            Y_: mnist.test.labels,
                                            keep_prob: 1.0}))
print("done")